In [1]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import losses
import torch
from torch import nn

import pandas as pd

In [3]:
sentence_model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2", 
                                     device='cuda')

for param in sentence_model.parameters():
    param.requires_grad = False
    
    
# Definir un nuevo módulo para las capas adicionales
class AdapterModule(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(AdapterModule, self).__init__()
        # Añade aquí más capas si necesitas
        self.dense = nn.Linear(input_dim, 1024)  # Ejemplo de dimensión intermedia
        self.output = nn.Linear(1024, output_dim)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.dense(x)
        x = self.activation(x)
        x = self.output(x)
        return x


class CustomDomainModel(nn.Module):
    def __init__(self, sentence_transformer, adapter):
        super(CustomDomainModel, self).__init__()
        self.sentence_transformer = sentence_transformer
        self.adapter = adapter

    def forward(self, input_ids, attention_mask):
        embeddings = self.sentence_transformer(input_ids, attention_mask=attention_mask)['sentence_embedding']
        output = self.adapter(embeddings)
        return output
    
    
# Determinar la dimensión de entrada de acuerdo con la última capa del modelo base
dim = sentence_model.get_sentence_embedding_dimension()

# Instancia del módulo adaptador
adapter = AdapterModule(dim, dim)

# Crear el modelo completo
custom_model = CustomDomainModel(sentence_model, adapter)

In [4]:
loss = losses.MultipleNegativesRankingLoss(sentence_model)


384

In [ ]:



# define evaluator
#from sentence_transformers.evaluation import InformationRetrievalEvaluator
# define over validation dataset

#evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)


In [8]:
data = pd.read_parquet("data/mlt_data_publications.parquet")

In [26]:
titles_map = {row.paperId:row.title for row in data.itertuples()}

In [29]:
related_pubs = [(titles_map[row.paperId], 'relatedWith', titles_map[row.source]) 
                for row in data.itertuples() if row.con_type!='base']

In [36]:
titles_embs = sentence_model.encode(list(titles_map.values()), show_progress_bar=True)

Batches:   0%|          | 0/412 [00:00<?, ?it/s]

In [39]:
titles_emb_map = {paperId:emb for paperId, emb in zip(titles_map, titles_embs)}

In [41]:
titles_embs[0].shape

(768,)

In [ ]:
embeddings = sentence_model.encode(df_reshaped['text_cln'].to_list(), show_progress_bar=True)

In [ ]:
# Iniciar el modelo MLP secuencial.
mlp_model = models.Sequential([
    # Aplanar la imagen para convertirla de una matriz 2D a un vector 1D.
    # El tamaño de entrada se ajusta al tamaño de las imágenes.

    # Primera capa densa con 1024 unidades.
    # BatchNormalization se utiliza para estabilizar el aprendizaje y se aplica antes de la activación.
    layers.Dense(1024),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.05),  # Dropout para la regularización.

    # Segunda capa densa con 512 unidades, con normalización y activación ReLU.
    layers.Dense(512),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.10),  # Un poco más de Dropout para esta capa más profunda.

    # Tercera capa densa con 256 unidades, y regularización L1 y L2 para evitar el sobreajuste.
    layers.Dense(256, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.15),

    # Cuarta capa densa con 128 unidades, también con regularización y BatchNormalization.
    layers.Dense(128, kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.20),

    # Capa de salida con activación softmax para la clasificación multiclase.
    layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compilar el modelo especificando el optimizador y la función de pérdida.
mlp_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['categorical_accuracy'])

# Callback para guardar el mejor modelo durante el entrenamiento, basado en la pérdida de validación.
checkpoint_callback = callbacks.ModelCheckpoint(
    filepath='models/mlp_best.keras',  # Guarda el modelo completo.
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)  # Solo guarda el modelo si es el mejor hasta el momento.

# Entrenar el modelo utilizando un generador de datos y validación con un conjunto de datos separados.
history_mlp = mlp_model.fit(
    train_generator,
    validation_data=(X_val_p, y_val_p),
    epochs=100,  # Número de épocas para entrenar.
    callbacks=[checkpoint_callback]  # Incluye el checkpoint para el guardado del modelo.
)

# Cargar el mejor modelo guardado después del entrenamiento.
mlp_model = models.load_model('models/mlp_best.keras')

NameError: name 'df_reshaped' is not defined